# r-3366-GUI

Taken on March 19th, Krypton run, Krypton trigger with "November 20162 conditions

In [ ]:
import datetime

In [ ]:
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
from __future__ import print_function
import sys
import os
from glob import glob
from time import time

In [ ]:
%gui qt
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import pandas as pd
import tables as tb
import numpy as np
import math


In [ ]:
from invisible_cities.database import load_db
import invisible_cities.sierpe.blr as blr
import invisible_cities.core.mpl_functions as mpl
import invisible_cities.reco.wfm_functions as wfm
import invisible_cities.reco.tbl_functions as tbl
import invisible_cities.reco.peak_functions_c as cpf
import invisible_cities.reco.peak_functions as pf
import invisible_cities.reco.pmaps_functions as pmf

In [ ]:
from invisible_cities.reco.pmap_io import pmap_writer, S12, S2Si

from   invisible_cities.core.configure import configure, print_configuration

from   invisible_cities.core.system_of_units_c import units
from   invisible_cities.cities.base_cities import PmapCity, SensorParams
from   invisible_cities.cities.base_cities import S12Params as S12P

In [ ]:
from invisible_cities.core.core_functions import define_window
from invisible_cities.core.core_functions import lrange
from collections import namedtuple

In [ ]:
from invisible_cities.reco.params import S12Params, ThresholdParams, CalibratedSum, PMaps, RawVectors,\
                                         SensorParams, CalibParams, PmapParams, CalibratedPMT, S1PMaps

In [ ]:
%load_ext Cython

In [ ]:
from matplotlib.figure import Figure

In [ ]:
from event_maps import Main

In [ ]:
from event_pmaps import pmt_calib_vectors, compare_S1, print_s12, compare_S1, S12F

In [ ]:
from plot_functions import plot_pmt_waveforms, plot_pmt_signals_vs_time_mus, \
                       plot_signal_vs_time_mus, plot_s12,  plot_vector, plot_xy, hist_1d
                       

### Import GUI

In [ ]:
from PyQt5 import QtCore, QtWidgets, uic
from matplotlib.backends.backend_qt5agg import (
    FigureCanvasQTAgg as FigureCanvas,
    NavigationToolbar2QT as NavigationToolbar)
from matplotlib.figure import Figure

from PyQt5.uic import loadUiType


qtCreatorFile = "event_maps.ui" # Enter file here.
Ui_MainWindow, QMainWindow = loadUiType(qtCreatorFile)

class Main(QMainWindow, Ui_MainWindow):
    def __init__(self, ):
        super(Main, self).__init__()
        self.setupUi(self)
        self.fig_dict = {}

        self.mplfigs.itemClicked.connect(self.change_figure)

        fig = Figure()
        self.add_mpl(fig)

    def add_figure(self, name, fig):
        test_key = self.fig_dict.pop(name, None)
        self.fig_dict[name] = fig
        if not test_key: # key not in dict
            self.mplfigs.addItem(name)

    def change_figure(self, item):
        text = item.text()
        self.rm_mpl()
        self.add_mpl(self.fig_dict[text])

    def add_mpl(self, fig):
        self.canvas = FigureCanvas(fig)
        self.mplvl.addWidget(self.canvas)
        self.canvas.draw()
        self.toolbar = NavigationToolbar(self.canvas,
                self.mplwindow, coordinates=True)
        self.mplvl.addWidget(self.toolbar)

    def rm_mpl(self,):
        self.mplvl.removeWidget(self.canvas)
        self.canvas.close()
        self.mplvl.removeWidget(self.toolbar)
        self.toolbar.close()


In [ ]:
main=Main()
main.show()

### Calibration vectors

In [ ]:
DataSiPM = load_db.DataSiPM()
adc_to_pes_sipm = DataSiPM.adc_to_pes.values
xs = DataSiPM.X.values
ys = DataSiPM.Y.values

In [ ]:
P = pmt_calib_vectors()

In [ ]:
P

## Files

In [ ]:
RWF_file = os.path.join(os.environ['IC_DATA'],'LSC/3366/dst_waves_f0_3366.root.h5')

In [ ]:
h5rwf = False

In [ ]:
if h5rwf:
    h5rwf.close()

In [ ]:
h5rwf = tb.open_file(RWF_file,'r')

### pmtrwf and sipmrwf vectors

To get vectors use **get_vectors(file)** in tbl_functions

In [ ]:
pmtrwf, pmtblr, sipmrwf = tbl.get_vectors(h5rwf)

In [ ]:
pmt_active = list(range(11))

In [ ]:
NEVT, NPMT, PMTWL = pmtrwf.shape
NEVT, NSIPM, SIPMWL = sipmrwf.shape
print("""
        Number of events in file = {}
        Number of PMTs = {}
        PMTWL = {}
        Number of SiPMs = {}
        SiPMWL = {}
      """.format(NEVT, NPMT, PMTWL,NSIPM, SIPMWL))

### Params

In [ ]:
s1par = S12Params(tmin=0*units.mus, tmax=640*units.mus, lmin=6, lmax=20, stride=4, rebin=False)

### Maps

In [ ]:
def event_maps(P, pmtrwf, sipmrwf, event=0):
    """ Events maps"""
    RWF = pmtrwf[event]
    CWF = blr.deconv_pmt(RWF, P.coeff_c, P.coeff_blr, n_baseline=48000, thr_trigger=5)
    CAL_PMT, CAL_PMT_MAU    =    cpf.calibrated_pmt_mau(CWF,
                                                        P.adc_to_pes,
                                                        pmt_active = P.pmt_active,
                                                        n_MAU = 100,
                                                        thr_MAU =  3)
    rms = [np.std(CAL_PMT[i]) for i in P.pmt_active]
    
    csum, csum_mau = cpf.calibrated_pmt_sum(CWF,P.adc_to_pes,P.pmt_active)
    s1par = S12Params(tmin=0*units.mus, tmax=640*units.mus, lmin=6, lmax=20, stride=4, rebin=False)
    s1_ene, s1_indx = cpf.wfzs(csum_mau, threshold=0.3)
    S1 =  cpf.find_S12(s1_ene, s1_indx, **s1par._asdict())
    print('***S1****')
    print_s12(S1)
    
    for i in S1:
        print('S1 number {}'.format(i))
        s1f = features_s12(S1[i])
        print('S1 features = {}'.format(s1f))
        t = S1[i][0]
        tmin = t[0] - 100*units.ns
        tmax = t[-1] + 100*units.ns
        
        #print('S1 match region: [tmin:tmax] (mus) = {}:{}'.format(tmin/units.mus,tmax/units.mus))
        s1par_PMT = S12Params(tmin=tmin, tmax=tmax, lmin=3, lmax=20, stride=4, rebin=False)
        
        PMT_S1 = {}
        for pmt in P.pmt_active:
            s1_ene, s1_indx = cpf.wfzs(CAL_PMT_MAU[pmt], threshold=0.1)
            PMT_S1[pmt] = cpf.find_S12(s1_ene, s1_indx, **s1par_PMT._asdict())
        nm = compare_S1(S1[i], PMT_S1)
        print('number of PMT matches = {}'.format(nm))
    
    s2_par = S12Params(tmin=640*units.mus, tmax=800*units.mus, stride=40, lmin=80, lmax=20000, rebin=True)
    s2_ene, s2_indx = cpf.wfzs(csum, threshold=1.0)
    S2    = cpf.find_S12(s2_ene, s2_indx, **s2_par._asdict())
   
    print_s12(S2)
    if len(S2) == 0:
        return 0
    
    s2f = features_s12(S2[0])
    print('S2 features = {}'.format(s2f))
    t = S2[0][0]
    tmin = t[-1] + 1*units.mus
    print('S1p search starts at {}'.format(tmin))
    s1p_params = S12Params(tmin=tmin, tmax=1300*units.mus, stride=4, lmin=4, lmax=20, rebin=False)
    s1_ene, s1_indx = cpf.wfzs(csum_mau, threshold=0.1)
    S1p =  cpf.find_S12(s1_ene, s1_indx, **s1p_params._asdict())
    S12t, S12l, S12e = scan_s12(S1p)
    
    if(len(S1) == 1 and len(S2) == 1):
        dt = s2f.tpeak - s1f.tpeak
        
        print('drif time = {} mus'.format(dt/units.mus))
    
    print('***S2Si****')
    sipm = cpf.signal_sipm(sipmrwf[event], adc_to_pes_sipm, thr=3.5*units.pes, n_MAU=100)
    SIPM = cpf.select_sipm(sipm)
    S2Si = pf.sipm_s2_dict(SIPM, S2, thr=30*units.pes)

    
    main.add_figure('RWF', plot_pmt_waveforms(RWF, zoom=False, window_size=10000))
    main.add_figure('CWF_vs_time_mus',
                plot_pmt_signals_vs_time_mus(CWF,
                                             P.pmt_active,
                                             t_min      =    0,
                                             t_max      = 1200,
                                             signal_min =    -5,
                                             signal_max =  200))
    main.add_figure('Calibrated_PMTs', 
                plot_pmt_signals_vs_time_mus(CAL_PMT,
                                             P.pmt_active,
                                             t_min      = 400,
                                             t_max      = 800,
                                             signal_min =  -2,
                                             signal_max =  10))
                    
    main.add_figure('PMT_rms',plot_xy(P.channel_id, rms))
    main.add_figure('Calibrated_SUM', 
                plot_signal_vs_time_mus(csum, t_min=0, t_max=1300, signal_min=-5, signal_max=60))
    main.add_figure('Calibrated_SUM_S1', 
                plot_signal_vs_time_mus(csum, t_min=0, t_max=640, signal_min=-2, signal_max=10))
    main.add_figure('Calibrated_SUM_S2', 
                plot_signal_vs_time_mus(csum, t_min=640, t_max=660, signal_min=-2, signal_max=100))
    main.add_figure('Calibrated_PMT_S1', 
                plot_pmt_signals_vs_time_mus(CAL_PMT,
                                             P.pmt_active,
                                             t_min      = 0,
                                             t_max      = 640,
                                             signal_min =  -2,
                                             signal_max =  2))
    if len(S1) > 0:
        main.add_figure('S1', plot_s12(S1))
    if len(S2) > 0:
        main.add_figure('S2', plot_s12(S2))
    main.add_figure('S1p_t',hist_1d(S12t, xlo=650*units.mus, xhi=1300*units.mus))
    main.add_figure('S1p_l',hist_1d(S12l, xlo=0, xhi=20))
    main.add_figure('S1p_e',hist_1d(S12e, xlo=0, xhi=10))
    pmf.scan_s2si_map(S2Si)


In [ ]:
event_maps(P, pmtrwf, sipmrwf,event=20)

In [ ]:
event=0
RWF = pmtrwf[event]

In [ ]:
main.add_figure('RWF', plot_pmt_waveforms(RWF, zoom=False, window_size=10000))

In [ ]:
CWF = blr.deconv_pmt(RWF, P.coeff_c, P.coeff_blr, n_baseline=48000, thr_trigger=5)

In [ ]:
main.add_figure('CWF', plot_pmt_waveforms(CWF, zoom=False, window_size=10000))

In [ ]:
main.add_figure('CWF_vs_time_mus',
                plot_pmt_signals_vs_time_mus(CWF,
                                             P.pmt_active,
                                             t_min      =    0,
                                             t_max      = 1200,
                                             signal_min =    -5,
                                             signal_max =  200))

In [ ]:
CAL_PMT, CAL_PMT_MAU    =    cpf.calibrated_pmt_mau(
                               CWF,
                               P.adc_to_pes,
                               pmt_active = list(range(11)),
                               n_MAU = 100,
                               thr_MAU =  3)

In [ ]:
rms = [np.std(CAL_PMT[i]) for i in P.pmt_active]

In [ ]:
rms

In [ ]:
P.channel_id

In [ ]:
main.add_figure('Calibrated_PMTs', 
                plot_pmt_signals_vs_time_mus(CAL_PMT,
                                             P.pmt_active,
                                             t_min      = 400,
                                             t_max      = 800,
                                             signal_min =  -2,
                                             signal_max =  10))

In [ ]:
main.add_figure('PMT_rms',plot_xy(P.channel_id, rms))

In [ ]:
csum, csum_mau = cpf.calibrated_pmt_sum(CWF,P.adc_to_pes,P.pmt_active)

In [ ]:
main.add_figure('Calibrated_SUM', 
                plot_signal_vs_time_mus(csum, t_min=0, t_max=1300, signal_min=-5, signal_max=60))

In [ ]:
main.add_figure('Calibrated_SUM_S1', 
                plot_signal_vs_time_mus(csum, t_min=550, t_max=600, signal_min=-2, signal_max=10))

In [ ]:
main.add_figure('Calibrated_SUM_S2', 
                plot_signal_vs_time_mus(csum, t_min=640, t_max=660, signal_min=-2, signal_max=100))

In [ ]:
main.add_figure('Calibrated_PMT_S1', 
                plot_pmt_signals_vs_time_mus(CAL_PMT,
                                             P.pmt_active,
                                             t_min      = 560,
                                             t_max      = 570,
                                             signal_min =  -2,
                                             signal_max =  2))

In [ ]:
main.add_figure('Calibrated_PMT_S2', 
                plot_pmt_signals_vs_time_mus(CAL_PMT,
                                             P.pmt_active,
                                             t_min      = 640,
                                             t_max      = 660,
                                             signal_min =  -2,
                                             signal_max =  10))

In [ ]:
s1par = S12Params(tmin=0*units.mus, tmax=620*units.mus, lmin=6, lmax=20, stride=4, rebin=False)

In [ ]:
s1_ene, s1_indx = cpf.wfzs(csum_mau, threshold=0.2)
S1 =  cpf.find_S12(s1_ene, s1_indx, **s1par._asdict())

In [ ]:
print_s12(S1)

In [ ]:
main.add_figure('S1', plot_s12(S1))

In [ ]:
S1[0][0]

In [ ]:
t = S1[0][0]

In [ ]:
tmin = t[0] - 100*units.ns
tmax = t[-1] + 100*units.ns

In [ ]:
s1par_PMT = S12Params(tmin=tmin, tmax=tmax, lmin=3, lmax=20, stride=4, rebin=False)

In [ ]:
PMT_S1 = {}
    
for pmt in list(range(11)):
    s1_ene, s1_indx = cpf.wfzs(CAL_PMT_MAU[pmt], threshold=0.1)
    PMT_S1[pmt] = cpf.find_S12(s1_ene, s1_indx, **s1par_PMT._asdict())

In [ ]:
PMT_S1

In [ ]:
compare_S1(S1, PMT_S1)

In [ ]:
s2_par = S12Params(tmin=620*units.mus, tmax=800*units.mus, stride=40, lmin=80, lmax=20000, rebin=True)

In [ ]:
s2_ene, s2_indx = cpf.wfzs(csum, threshold=0.5)
S2              = cpf.find_S12(s2_ene, s2_indx, **s2_par._asdict())

In [ ]:
print_s12(S2)

In [ ]:
main.add_figure('S2', plot_s12(S2))

In [ ]:
t = S2[0][0]
tmin = t[-1] + 1*units.mus

In [ ]:
s1p_params = S12Params(tmin=tmin, tmax=1300*units.mus, stride=4, lmin=4, lmax=20, rebin=False)

In [ ]:
s1_ene, s1_indx = cpf.wfzs(csum_mau, threshold=0.1)
S1p =  cpf.find_S12(s1_ene, s1_indx, **s1p_params._asdict())

In [ ]:
S12t, S12l, S12e = scan_s12(S1p)

In [ ]:
S12t/units.mus

In [ ]:
S12l

In [ ]:
S12e

In [ ]:
hist

In [ ]:
bin_edges

In [ ]:
main.add_figure('S1p_t',hist_1d(S12t/units.mus, xlo=S12t[0]/units.mus, xhi=S12t[-1]/units.mus))

In [ ]:
main.add_figure('S1p_l',hist_1d(S12l, xlo=0, xhi=20))

In [ ]:
main.add_figure('S1p_e',hist_1d(S12e, xlo=0, xhi=10))

In [ ]:
sipm = cpf.signal_sipm(sipmrwf[event], adc_to_pes_sipm, thr=3.5*units.pes, n_MAU=100)
SIPM = cpf.select_sipm(sipm)
S2Si = pf.sipm_s2_dict(SIPM, S2, thr=30*units.pes)


In [ ]:
S2Si

In [ ]:
S2

In [ ]:
pmf.scan_s2si_map(S2Si)


In [ ]:
xs[1688]

In [ ]:
xs[1689]

In [ ]:
ys[1688]

In [ ]:
ys[1689]

In [ ]:

def event_maps(P, pmtrwf, sipmrwf, s1par, thr_s1=0.3*units.pes, event=0):

    main.add_figure('RWF', plot_pmt_waveforms(RWF, zoom=False, window_size=10000))
    main.add_figure('CWF_vs_time_mus',
                plot_pmt_signals_vs_time_mus(CWF,
                                             P.pmt_active,
                                             t_min      =    0,
                                             t_max      = 1200,
                                             signal_min =    -5,
                                             signal_max =  200))
    main.add_figure('Calibrated_PMTs',
                plot_pmt_signals_vs_time_mus(CAL_PMT,
                                             P.pmt_active,
                                             t_min      = 400,
                                             t_max      = 800,
                                             signal_min =  -2,
                                             signal_max =  10))

    main.add_figure('PMT_rms',plot_xy(P.channel_id, rms))
    main.add_figure('Calibrated_SUM',
                plot_signal_vs_time_mus(csum, t_min=0, t_max=1300, signal_min=-5, signal_max=60))
    main.add_figure('Calibrated_SUM_S1',
                plot_signal_vs_time_mus(csum, t_min=0, t_max=640, signal_min=-2, signal_max=10))
    main.add_figure('Calibrated_SUM_S2',
                plot_signal_vs_time_mus(csum, t_min=640, t_max=660, signal_min=-2, signal_max=100))
    main.add_figure('Calibrated_PMT_S1',
                plot_pmt_signals_vs_time_mus(CAL_PMT,
                                             P.pmt_active,
                                             t_min      = 0,
                                             t_max      = 640,
                                             signal_min =  -2,
                                             signal_max =  2))
    if len(S1) > 0:
        main.add_figure('S1', plot_s12(S1))
    if len(S2) > 0:
        main.add_figure('S2', plot_s12(S2))
    main.add_figure('S1p_t',hist_1d(S12t, xlo=650*units.mus, xhi=1300*units.mus))
    main.add_figure('S1p_l',hist_1d(S12l, xlo=0, xhi=20))
    main.add_figure('S1p_e',hist_1d(S12e, xlo=0, xhi=10))
    pmf.scan_s2si_map(S2Si)

